# BRAIN TUMOR DETECTION USING DEEP LEARNING

#### Dataset used : https://www.kaggle.com/sartajbhuvaji/brain-tumor-classification-mri#

# EDA of the dataset

## Importing libraries

In [2]:
#to ignore the warnings
import warnings
warnings.filterwarnings("ignore")

#for extracting zip file dataset 
import zipfile

import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import scipy
import glob
import os
import shutil
import math
import imutils
import cv2

from tensorflow.keras.preprocessing import image_dataset_from_directory

#for building CNN model
from keras.layers import Dense, Conv2D, Flatten, MaxPool2D, Dropout, BatchNormalization, MaxPooling2D,GlobalAvgPool2D
from keras.models import Sequential
from keras.preprocessing import image
import keras

## Reading zip file

In [29]:
with zipfile.ZipFile("archive.zip","r") as zip_ref:
    zip_ref.extractall()

In [3]:
test=image_dataset_from_directory("Testing",label_mode="categorical",image_size=(300,300),batch_size=100)
train=image_dataset_from_directory("Training",label_mode="categorical",image_size=(300,300),batch_size=100)

Found 394 files belonging to 4 classes.
Found 2870 files belonging to 4 classes.


# CNN MODEL

In [58]:
model = Sequential()
model.add(Conv2D(36,(12,12), activation='relu', input_shape = (300,300,3)))

model.add(Conv2D(64,(6,6), activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(128,(4,4), activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(256,(3,3), activation='relu' ))
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Dropout(rate=0.1))

model.add(Flatten())
model.add(Dense(units= 100, activation='relu'))
model.add(Dropout(rate=0.1))
model.add(Dense(40,activation='relu'))
model.add(Dense(units= 4, activation='softmax'))

model.compile(loss = keras.losses.binary_crossentropy, optimizer = 'adam', metrics=['accuracy'])

model.summary()


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_12 (Conv2D)           (None, 289, 289, 36)      15588     
_________________________________________________________________
conv2d_13 (Conv2D)           (None, 284, 284, 64)      83008     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 142, 142, 64)      0         
_________________________________________________________________
conv2d_14 (Conv2D)           (None, 139, 139, 128)     131200    
_________________________________________________________________
max_pooling2d_10 (MaxPooling (None, 69, 69, 128)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 67, 67, 256)       295168    
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 33, 33, 256)      

# Preparing our data using data Generator

In [5]:
train_datagen = image.ImageDataGenerator(
    zoom_range = 0.2, shear_range = 0.2, rescale = 1./255, horizontal_flip = True
)

test_datagen = image.ImageDataGenerator(rescale = 1./255)



In [44]:
train_data = train_datagen.flow_from_directory(directory='Training', target_size= (300,300), batch_size=64,
class_mode = 'binary'
)

Found 2870 images belonging to 4 classes.


In [45]:
train_data.class_indices

{'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}

In [46]:
test_data = train_datagen.flow_from_directory(directory='Testing', target_size= (300,300,3), batch_size=64,
class_mode = 'binary'
)

Found 394 images belonging to 4 classes.


In [47]:
test_data.class_indices

{'glioma_tumor': 0, 'meningioma_tumor': 1, 'no_tumor': 2, 'pituitary_tumor': 3}

# Early Stopping and Check Point

In [63]:
from keras.callbacks import ModelCheckpoint, EarlyStopping

#early stopping
es = EarlyStopping(monitor = 'accuracy', min_delta = 0.01, patience = 5, verbose = 1, mode = 'auto')

#model checkpoint
mc = ModelCheckpoint(filepath = "best_model.h5", monitor = 'accuracy', verbose = 1, save_best_only = True, mode = 'auto')

# putting call back in a list
call_back = [es , mc]

In [65]:
hist = model.fit_generator(generator= train , 
epochs = 15, 
validation_data = test,
callbacks = call_back)

Epoch 1/15
29/29 [==============================] - 1383s 48s/step - loss: 0.1983 - accuracy: 0.8568 - val_loss: 0.7610 - val_accuracy: 0.6142

Epoch 00001: accuracy improved from 0.82979 to 0.85679, saving model to best_model.h5
Epoch 2/15
29/29 [==============================] - 1377s 48s/step - loss: 0.1337 - accuracy: 0.9125 - val_loss: 0.7275 - val_accuracy: 0.6675

Epoch 00002: accuracy improved from 0.85679 to 0.91254, saving model to best_model.h5
Epoch 3/15
29/29 [==============================] - 1378s 48s/step - loss: 0.0896 - accuracy: 0.9460 - val_loss: 0.8461 - val_accuracy: 0.7157

Epoch 00003: accuracy improved from 0.91254 to 0.94599, saving model to best_model.h5
Epoch 4/15
29/29 [==============================] - 1378s 48s/step - loss: 0.0616 - accuracy: 0.9683 - val_loss: 1.1662 - val_accuracy: 0.7056

Epoch 00004: accuracy improved from 0.94599 to 0.96829, saving model to best_model.h5
Epoch 5/15
29/29 [==============================] - 1380s 48s/step - loss: 0.049